<a href="https://colab.research.google.com/github/BenjamimOliveira/CN_TP1/blob/main/script/Script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1 - CN


## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import tensorflow.keras.optimizers as optimizers
import time
import zipfile
import random

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load data sources

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

start_time = time.time()

# Copy data (.zip file)
!cp -r 'drive/MyDrive/UMinho/CN/dataset.zip' 'dataset.zip'

# Extract .zip file
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset_extr')

print("%s segundos" % ((time.time() - start_time)))

Mounted at /content/drive
33.83084487915039 minutos


## Data Loading

In [3]:
train_dir = '/content/dataset_extr/dataset/train'
test_dir = '/content/dataset_extr/dataset/test'
valid_dir = '/content/dataset_extr/dataset/valid'

imgDataGenerator = False

if imgDataGenerator:
  train_dtgen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
  test_dtgen = ImageDataGenerator(rescale = 1./255)
  valid_dtgen = ImageDataGenerator(rescale = 1./255)

  train_ds = train_dtgen.flow_from_directory(train_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  test_ds = test_dtgen.flow_from_directory(test_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  valid_ds = valid_dtgen.flow_from_directory(valid_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
else:
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      train_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
      test_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
      valid_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  
  class_names = train_ds.class_names

  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 35215 files belonging to 250 classes.
Found 1250 files belonging to 250 classes.
Found 1250 files belonging to 250 classes.


## Algoritmo Genético

In [4]:
##
# Formato do gene [nParesConv2DMaxPooling(entre 1 e 3), learningRate, Momentum, Nesterov, batchSize, Epochs, Accuracy]
#
##



### Gene Aleatório

In [6]:
def random_element():
  gene = []
  
  # -- nPares Conv2D/MaxPooling
  gene.append(random.randint(1,3))

  # -- Learning rate
  gene.append(random.randint(0,4))

  # -- Momentum
  gene.append(random.randint(0, 9)/10)

  # -- Nesterov
  gene.append(random.randint(0,1))

  # -- BatchSize
  gene.append(random.randint(batchSize[0],batchSize[1]))

  # -- Epochs
  gene.append(random.randint(epochs[0], epochs[1]))

  # -- Accuracy
  gene.append(-1)

  return gene

### População Aleatória/Inicial

In [7]:
def random_pool_generator(poolSize):
  pool = []

  for x in range(poolSize):
    pool.append(random_element())

  return pool

### Selecionar "pais" da próxima geração

In [38]:
def select_mating_pool(poolSize, pool):
  top = round(poolSize/2)
  def orderBy(a):
    return a[6]

  pool.sort(reverse=True, key=orderBy)
  return pool[0:top]
  

### Crossover entre 2 elementos

In [89]:
def mix2elements(elemento1, elemento2):
  # tamanho do gene
  print("Elemento:")
  print(type(elemento1))
  print(elemento1)
  size = len(elemento1) - 1
  # elementos que irão ser cruzados
  crossElements = []
  for x in range(3):
    rand = random.randint(0, size-1)
    while (rand in crossElements):
      rand = random.randint(0, size-1)
    crossElements.append(rand)

  elementoFilho = []
  aux = 0

  while aux < size:
    if aux in crossElements:
      elementoFilho.append(elemento1[aux])
    else:
      elementoFilho.append(elemento2[aux])
    aux += 1
  # indica que ainda não foi treinado  
  elementoFilho.append(-1)
  return elementoFilho

### Selecionar que elementos vão fazer crossover

In [91]:
def crossover(poolSize, pool):
  currentPool = poolSize

  # mistura o primeiro com o último
  pool.append(mix2elements(pool[0], pool[-1]))

  aux = 0
  for i in range(int(round(poolSize/2))-1):
    pool.append(mix2elements(pool[aux], pool[aux+1]))
  return pool

### Mutação de uma geração

In [ ]:
def mutation(pool, chance):
  chance = chance * 100
  print(chance)

### Gerador de CNN (segundo o gene)

In [12]:
def model_generator(nPares):
  mod = []
  mod.append(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(64, 64, 3)))
  if nPares >= 1: 
    mod.append(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if nPares >= 2:
    mod.append(layers.Conv2D(32, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if nPares >= 3:
    mod.append(layers.Conv2D(64, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  mod.append(layers.Flatten())
  mod.append(layers.Dense(128, activation='relu'))
  mod.append(layers.Dense(num_classes))
  model = Sequential(mod)
  return model

### Conversor de gene em acurácia (treino da cnn com o gene)

In [16]:
def gene_converter(gene):
  # -- MODELO
  model = model_generator(gene[0])
  # -- COMPILAR MODELO
  opt = optimizers.SGD(learning_rate=0.01, momentum=0.9)
  # Experimentar loss='sparse_categorical_crossentropy'
  if imgDataGenerator:
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  else:
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer='adam',
                loss=loss,
                metrics=['accuracy'])
  # -- TREINAR MODELO
  epochs=1
  start_time = time.time()
  with tf.device('/device:GPU:0'):
    history = model.fit(
      train_ds,
      validation_data=valid_ds,
      epochs=epochs
    )
  # -- AVALIAR MODELO
  results = model.evaluate(test_ds)
  return results[1]

### Algoritmo Final

In [92]:
# --------- Parametros chave ---------
learningRate = [1, 0.1, 0.01, 0.001, 0.0001]
batchSize = [8, 128]
epochs = [4, 20]
num_classes = 250
poolSize = 4
mutationChance = 0.2
# ------------------------------------

pool = random_pool_generator(poolSize)
print(pool)

maximiza = []

# treina a geração
for gene in pool:
  # -- se ainda não tiver sido treinado numa geração passada então treina
  if gene[6] == -1:
    gene[6] = gene_converter(gene=gene)
  
# seleciona metade do poolsize para se reproduzir
pool = select_mating_pool(poolSize, pool)
pool = crossover(poolSize, pool)
pool = mutation(pool, mutationChance)

[[3, 2, 0.9, 1, 38, 5, -1], [3, 1, 0.9, 1, 99, 11, -1], [3, 3, 0.4, 0, 114, 4, -1], [1, 3, 0.0, 1, 109, 18, -1]]
1101/1101 [==============================] - 5s 4ms/step - loss: 5.1493 - accuracy: 0.0365 - val_loss: 3.2161 - val_accuracy: 0.2560
0.08097312450408936 minutos
1101/1101 [==============================] - 5s 4ms/step - loss: 5.0547 - accuracy: 0.0450 - val_loss: 3.0737 - val_accuracy: 0.2832
0.07739024957021078 minutos
1101/1101 [==============================] - 5s 4ms/step - loss: 5.1226 - accuracy: 0.0354 - val_loss: 3.2870 - val_accuracy: 0.2512
0.07831404209136963 minutos
1101/1101 [==============================] - 4s 3ms/step - loss: 5.5323 - accuracy: 0.0068 - val_loss: 5.5235 - val_accuracy: 0.0040
0.06558732986450196 minutos
40/40 [==============================] - 0s 2ms/step - loss: 5.5235 - accuracy: 0.0040
Elemento:
<class 'list'>
[3, 1, 0.9, 1, 99, 11, 0.31040000915527344]
Elemento:
<class 'list'>
[3, 1, 0.9, 1, 99, 11, 0.31040000915527344]


In [93]:
#for x in pool:
#  print(x)
#print(len(pool[0]))
#print(random.randint(0,5))
#elemento1 = [1,1,1,1,1,1,-1]
#elemento2 = [2,2,2,2,2,2,-1]
#print(len(elemento1)-1)
print(pool)
pool = mutation(pool, 0.2)

[[3, 1, 0.9, 1, 99, 11, 0.31040000915527344], [3, 3, 0.4, 0, 114, 4, 0.2671999931335449], [3, 1, 0.9, 1, 114, 4, -1], [3, 3, 0.9, 0, 99, 11, -1]]
